# Notebook 3: Lowest Order Rising Bubble

This notebook will demonstrate the use of the lowest-order finite element spaces. To increase the order of accuracy of the transport schemes we use a recovered finite element method where the transported fields are recovered into a higher-order function space before the transport is performed.

We will demonstrate how to set this up to simulate a rising bubble, using the dry test of [Bryan and Fritsch (2002)](https://journals.ametsoc.org/view/journals/mwre/130/12/1520-0493_2002_130_2917_absfmn_2.0.co_2.xml).

As usual, we begin by importing the required functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (IntervalMesh, ExtrudedMesh,
                       SpatialCoordinate, conditional, cos, pi, sqrt,
                       TestFunction, dx, TrialFunction, Constant, Function,
                       LinearVariationalProblem, LinearVariationalSolver)

INFO     Running /Users/JS1075/firedrake_dec24/lib/python3.12/site-packages/ipykernel_launcher.py -f /private/var/folders/f0/llvlmlb50qg6mmlxs8m6d6d00000gp/T/tmpziop690p.json --HistoryManager.hist_file=:memory:


We now define the time-step size and simulation time.

In [2]:
dt = 1.0
tmax = 1000.

We will contruct our two dimensional mesh by extruding a 1D `IntervalMesh`. Here we specify the width and height of the mesh and the grid size `deltax` which is the same in both the x and z directions. We set up the lowest order function spaces by specifying `degree=0` when setting up the domain. 

In [3]:
domain_width = 10000.
domain_height = 10000.
deltax = 100.
nlayers = int(domain_height/deltax)
ncolumns = int(domain_width/deltax)
m = IntervalMesh(ncolumns, domain_width)
mesh = ExtrudedMesh(m, layers=nlayers, layer_height=domain_height/nlayers)
domain = Domain(mesh, dt, family='CG', degree=0)

This setup uses the default values for the relevant physical parameters (which can be found [here](https://www.firedrakeproject.org/gusto-docs/gusto.core.html#gusto.core.configuration.CompressibleParameters)). We will solve the compressible Euler equations with the vector advection form of the velocity transport term and, in addition to having no flow through the top and bottom boundaries, we also specify no flow through the side boundaries.

In [4]:
params = CompressibleParameters()
eqns = CompressibleEulerEquations(domain, params, u_transport_option='vector_advection_form',
                                  no_normal_flow_bc_ids=[1, 2])

TypeError: EquationParameters.__init__() missing 1 required positional argument: 'mesh'

The following cell sets up the output, including a diagnostic field to allow us to visualise the rising thermal.

In [5]:
dirname = 'lowest_order_bubble'
output = OutputParameters(dirname=dirname, dumplist=['u'])
diagnostic_fields = [Perturbation('theta')]
io = IO(domain, output, diagnostic_fields=diagnostic_fields)

We now setup the transport schemes, including the options required for the recovery scheme. These are passed in to the `SemiImplicitQuasiNewton` timestepper, along with the linear solver.

In [6]:
boundary_methods = {'DG': BoundaryMethod.taylor,
                    'HDiv': BoundaryMethod.taylor}

recovery_spaces = RecoverySpaces(domain, boundary_methods, use_vector_spaces=True)

u_opts = recovery_spaces.HDiv_options
rho_opts = recovery_spaces.DG_options
theta_opts = recovery_spaces.theta_options

transported_fields = [
    SSPRK3(domain, "rho", options=rho_opts),
    SSPRK3(domain, "theta", options=theta_opts),
    SSPRK3(domain, "u", options=u_opts)
]

transport_methods = [DGUpwind(eqns, field) for field in ["u", "rho", "theta"]]

linear_solver = CompressibleSolver(eqns)

stepper = SemiImplicitQuasiNewton(eqns, io, transported_fields, transport_methods,
                                  linear_solver=linear_solver)

NameError: name 'eqns' is not defined

We can now specify the background potential temperature and density fields, which are in hydrostatic balance.

In [7]:
rho0 = stepper.fields("rho")
theta0 = stepper.fields("theta")

Vt = domain.spaces("theta")
Vr = domain.spaces("DG")

xz = SpatialCoordinate(mesh)
Tsurf = 300.0
theta_b = Function(Vt).interpolate(Constant(Tsurf))

# Calculate hydrostatic fields
compressible_hydrostatic_balance(eqns, theta_b, rho0, solve_for_rho=True)

# make mean fields
rho_b = Function(Vr).assign(rho0)

stepper.set_reference_profiles([('rho', rho_b), ('theta', theta_b)])

NameError: name 'stepper' is not defined

Now we define the bubble perturbation in terms of the potential temperature and find the corresponding density perturbation.

In [8]:
xc = domain_width / 2     # x position of bubble centre
zc = 2000.                # z position of bubble centre
rc = 2000.                # radius of bubble
Tdash = 2.0               # magnitude of bubble perturbation

# define bubble perturbation
r = sqrt((xz[0] - xc) ** 2 + (xz[1] - zc) ** 2)
theta_pert = Function(Vt).interpolate(
    conditional(r > rc, 0.0, Tdash * (cos(pi * r / (2.0 * rc))) ** 2)
)

# define initial theta
theta0.interpolate(theta_b * (theta_pert / 300.0 + 1.0))

# find perturbed rho
gamma = TestFunction(Vr)
rho_trial = TrialFunction(Vr)
lhs = gamma * rho_trial * dx
rhs = gamma * (rho_b * theta_b / theta0) * dx
rho_problem = LinearVariationalProblem(lhs, rhs, rho0)
rho_solver = LinearVariationalSolver(rho_problem)
rho_solver.solve()

NameError: name 'xz' is not defined

Now, we can run the simulation. The cell below runs for just two timesteps because the full simulation takes a while, but if you replace the `2*dt` below with `tmax`, which was defined above to be 1000s, you should be able to reproduce something like the plots below.

In [9]:
stepper.run(t=0, tmax=2*dt)

NameError: name 'stepper' is not defined

The plot below shows the initial and final potential temperature perturbation.

![bubble](../figures/compressible_euler/dry_bryan_fritsch.png)